In [ ]:
import tensorflow as tf
#tf.test.gpu_device_name()
from google.colab import drive

# 使用工具colab的接口挂载google drive目录，这样可以从外部获取数据并且可以把训练好的模型保存在google drive上
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip3 install SpeechRecognition
!pip3 install wheel
!sudo apt-get install python python-all-dev python-pip build-essential swig git libpulse-dev libasound2-dev
!pip3 install pocketsphinx
!pip3 install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def get_bert(new_translation):
  #bert_embedding = BertEmbedding(),
  test=[new_translation]
  test_text_embed = model.encode(test)
  return test_text_embed

def bert_simi(ori_bert, new_bert):
  return cosine_similarity(ori_bert, new_bert)


In [ ]:
import speech_recognition as speech

import speech_recognition as speech

def transcribe_google(audiopath):

  recognizer = speech.Recognizer()
  harvard = speech.AudioFile(audiopath)  # 读取音频文件
  with harvard as source:
      audio = recognizer.record(source)
  # 使用google识别语音
  try:
      #print("google识别结果为:" + recognizer.recognize_google(audio))
      #换一下sphinx
      return recognizer.recognize_google(audio)
  except speech.UnknownValueError:
      pass
  except speech.RequestError as e:
      pass

In [ ]:
#暂时用一下，后面肯定要再改
import scipy.io.wavfile as wav
import copy
def store_new(flag,fs,x,path_store):
  data_final=copy.deepcopy(x)
  for i in range(len(flag)):
    if(flag[i]==0):
      data_final[512*i:512*(i+1)]=copy.deepcopy(512*[0])
  
  wav.write(path_store, fs, data_final)

In [ ]:
from sys import flags
import scipy.io.wavfile as wav
import pandas as pd
import os
import wave
import numpy as np
import copy
import random
#add a function to compare simialrity and get a true and flase and one score
def compare_tfidf(key, new_translation):

#比较tf-idf的时候需要的是新的一句翻译，然后对它进行分词，得到一个分词列表，然后通过key去查阅对应的字典及其概率，然后相加，得到正确或者错误以及extent
#需要：key,分词算法，translation
#return: boolean 和一个score
  if new_translation== None:
    return False 
  dic_=np.load('/content/gdrive/MyDrive/play_deepcover/translation/commonvoice_tf_idf_normalized.npy',allow_pickle=True)
  tfidf_all=dic_.item()
  tfidf_ori=tfidf_all[key]
  word=new_translation.strip().split()
  words=list(set(word))
  score=0
  for i in range (len(words)):
    if(words[i] in tfidf_ori.keys()):
      score+=tfidf_ori[words[i]]

  if(score>0.5):
    score=(score-0.5)/0.5
    return True
  else:
    score=(0.5-score)/0.5
    return False


def compare_bert(key, new_translation):
  if new_translation== None:
    return False
  dic_=np.load('/content/gdrive/MyDrive/play_deepcover/translation/bert_commonvoice.npy',allow_pickle=True)
  bert_all=dic_.item()
  ori_bert=bert_all[key]
  new_bert=get_bert(new_translation)
  score=bert_simi(ori_bert, new_bert)[0][0]
  print("========================================")
  print(score)
  if(score>0.5):
    score=(score-0.5)/0.5
    return True
  else:
    score=(0.5-score)/0.5
    return False

#需要：key,bertembeding算法，translation
#return: boolean 和一个score


In [ ]:
!pip3 install lime

In [ ]:
!pip3 install jiwer

     |████████████████████████████████| 50 kB 3.0 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149866 sha256=a6a885f4e7a88181b7f3e52da5abedd457ef12570c65c7c4877095368488ad93
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [ ]:
from jiwer import wer

def classifier_ori_bert(key,audio):
    
    newaudio_y=transcribe_deepspeech(audio)
    classname=compare_bert(key,newaudio_y)
    return classname

def classifier_ori_tfidf(key,audio):
    
    newaudio_y=transcribe_deepspeech(audio)
    classname=compare_tfidf(key,newaudio_y)
    return classname

def classifier(key,audio):

    
    newaudio_y=transcribe_deepspeech(audio)
    y=transcription_list[key]
    if(newaudio_y==None or y==None):
      return False
    l=wer(y,newaudio_y)
    if(l==0.0):
      classname=True
    else:
      classname=False
    

    return classname


In [ ]:
from numpy.random.mtrand import permutation
import numpy as np
import sklearn
from lime import explanation
from lime import lime_base
import math
import logging

class TSDomainMapper(explanation.DomainMapper):
    def __init__(self,l):
        """Init function.
        Args:
            signal_names: list of strings, names of signals
        """
        pass
        
    def map_exp_ids(self, exp, **kwargs):
        # in case of univariate, don't change feature ids
        return exp
        

class LimeTimeSeriesExplainer(object):
    """Explains time series classifiers."""

    def __init__(self,
                 kernel_width=25,
                 verbose=False,
                 class_names=None,
                 feature_selection='auto',
                 #signal_names=["not specified"]
                 ):
        """Init function.
        Args:
            kernel_width: kernel width for the exponential kernel
            verbose: if true, print local prediction values from linear model
            class_names: list of class names, ordered according to whatever the
            classifier is using. If not present, class names will be '0',
                '1', ...
            feature_selection: feature selection method. can be
                'forward_selection', 'lasso_path', 'none' or 'auto'.
            signal_names: list of strings, names of signals
        """

        # exponential kernel
        def kernel(d): return np.sqrt(np.exp(-(d ** 2) / kernel_width ** 2))

        self.base = lime_base.LimeBase(kernel, verbose)
        self.class_names = class_names
        self.feature_selection = feature_selection
        #self.signal_names = signal_names

    def explain_instance(self,
                         timeseries_instance,
                         classifier_fn,
                         key=None,
                         num_samples=10,
                         model_regressor=None,
                         replacement_method='mean'):
        """Generates explanations for a prediction.
        First, we generate neighborhood data by randomly hiding features from
        the instance (see __data_labels_distance_mapping). We then learn
        locally weighted linear models on this neighborhood data to explain
        each of the classes in an interpretable way (see lime_base.py).
        As distance function DTW metric is used.
        Args:
            time_series_instance: time series to be explained.
            classifier_fn: classifier prediction probability function,
                which takes a list of d arrays with time series values
                and outputs a (d, k) numpy array with prediction
                probabilities, where k is the number of classes.
                For ScikitClassifiers , this is classifier.predict_proba.
            num_slices: Defines into how many slices the time series will
                be split up
            labels: iterable with labels to be explained.
            top_labels: if not None, ignore labels and produce explanations for
            the K labels with highest prediction probabilities, where K is
            this parameter.
            num_features: maximum number of features present in explanation
            num_samples: size of the neighborhood to learn the linear model
            distance_metric: the distance metric to use for sample weighting,
                defaults to cosine similarity
            model_regressor: sklearn regressor to use in explanation. Defaults
                to Ridge regression in LimeBase. Must have
                model_regressor.coef_ and 'sample_weight' as a parameter to
                model_regressor.fit()
        Returns:
            An Explanation object (see explanation.py) with the corresponding
            explanations.
       """
        print(key,classifier_fn)

        f_n=int(len(timeseries_instance)/512)
        permutations, predictions, distances = self.__data_labels_distances(
            timeseries_instance, key,classifier_fn,
            num_samples, f_n, replacement_method)

        #is_multivariate = len(timeseries_instance.shape) > 1
        
        #if self.class_names is None:
         #   self.class_names = [str(x) for x in range(predictions[0].shape[0])]

        domain_mapper = TSDomainMapper(self)
        #ret_exp = explanation.Explanation(domain_mapper=domain_mapper,
         #                                 class_names=self.class_names)
        ret_exp = explanation.Explanation(domain_mapper=domain_mapper,class_names=self.class_names)
        ret_exp.predict_proba = 1

        labels=[1,0]
        permutations=np.array(permutations)
        
        predictions=np.array(predictions).reshape(num_samples,)


        x_1=np.array([0,1])
        x_2=np.array([1,0])

        all_pre=[]
        for i in range(len(predictions)):
          if(predictions[i]==1):
            all_pre.append(x_1)
          else:
            all_pre.append(x_2)
        distances=np.array(distances)
        print(np.array(all_pre).shape)
        print(distances.shape)

        for i in range(0,2):
            (ret_exp.intercept[int(i)],
             ret_exp.local_exp[int(i)],
             ret_exp.score,
             ret_exp.local_pred) = self.base.explain_instance_with_data(
                permutations, np.array(all_pre),
                distances, i,
                f_n,
                model_regressor=model_regressor,
                feature_selection=self.feature_selection)
        '''
        ret_exp.predict_proba = predictions[0]

        if top_labels:
            labels = np.argsort(predictions[0])[-top_labels:]
            ret_exp.top_labels = list(predictions)
            ret_exp.top_labels.reverse()
        for label in labels:
            (ret_exp.intercept[int(label)],
             ret_exp.local_exp[int(label)],
             ret_exp.score,
             ret_exp.local_pred) = self.base.explain_instance_with_data(
                permutations, predictions,
                distances, label,
                num_features,
                model_regressor=model_regressor,
                feature_selection=self.feature_selection)
        '''
        return ret_exp

    def __data_labels_distances(cls,
                                timeseries,
                                key,
                                classifier_fn,
                                num_samples,
                                f_n,
                                replacement_method='mean'):
        """Generates a neighborhood around a prediction.
        Generates neighborhood data by randomly removing slices from the
        time series and replacing them with other data points (specified by
        replacement_method: mean over slice range, mean of entire series or
        random noise). Then predicts with the classifier.
        Args:
            timeseries: Time Series to be explained.
                it can be a flat array (univariate)
                or (num_signals, num_points) (multivariate)
            classifier_fn: classifier prediction probability function, which
                takes a time series and outputs prediction probabilities. For
                ScikitClassifier, this is classifier.predict_proba.
            num_samples: size of the neighborhood to learn the linear
                model (perturbation + original time series)
            num_slices: how many slices the time series will be split into
                for discretization.
            replacement_method:  Defines how individual slice will be
                deactivated (can be 'mean', 'total_mean', 'noise')
        Returns:
            A tuple (data, labels, distances), where:
                data: dense num_samples * K binary matrix, where K is the
                    number of slices in the time series. The first row is the
                    original instance, and thus a row of ones.
                labels: num_samples * L matrix, where L is the number of target
                    labels
                distances: distance between the original instance and
                    each perturbed instance
        """

        def distance_fn(x,flag_):
          similarity=[]
          print(np.array(x))
          print(np.array(flag_))


          for i in range(len(x)):
            print(cosine_similarity(np.array(x[i]).reshape(1, len(x[i])),np.array(flag_).reshape(1, len(flag_))))
            
            similarity.append(cosine_similarity(np.array(x[i]).reshape(1, len(x[i])),np.array(flag_).reshape(1, len(flag_)))[0][0]*100)
          return similarity

            #return sklearn.metrics.pairwise.pairwise_distances(
                #x, timeseries.reshape(-1, 1), metric='cosine').ravel() * 100


        len_ts=len(timeseries)
        k=0

        samples=[]
        num=int(0.05*f_n)
        if(num<1):
          num=1

        
        print(num_samples)
        flag_=[1]*f_n
        flag_all=[]

        for i in range(num_samples):
          #f=np.random.randint(1, f_n + 1, num_samples - 1)
          f=np.random.randint(0, f_n, num)
          flag_n=copy.deepcopy(flag_)
          purturbation=timeseries.copy().tolist()
          for j in range(len(f)):
            purturbation[f[j]*512:f[j]*512+512]=copy.deepcopy([0]*512)
            flag_n[f[j]]=0
          
          
          flag_all.append(flag_n)

          samples.append(purturbation)

        
        predictions=[]


        for i in range(num_samples):
          path="/content/gdrive/MyDrive/play_deepcover/A_temp/"+key+".wav"
          data=np.array(samples[i]).astype(np.int16)
          wav.write(path, 16000, data)
          prediction=classifier_fn(key,path)
          print(prediction)
          if(prediction==True):
            predictions.append(1)
          if(prediction==False):
            predictions.append(0)

        distances = distance_fn(flag_all,flag_)

        return flag_all, predictions, distances


In [ ]:

# divide audios into frames and give them labels(all 1) 

import scipy.io.wavfile as wav
all_=[]
path_x="sample.wav"
fs, x = wav.read(path_x)
all_.append(x)



In [ ]:
print(all_)

{'sample-000000': array([ 0,  0,  0, ...,  3, 25, 23], dtype=int16), 'sample-000001': array([0, 0, 0, ..., 0, 0, 0], dtype=int16), 'sample-000002': array([0, 0, 0, ..., 0, 0, 0], dtype=int16), 'sample-000003': array([0, 0, 0, ..., 0, 0, 0], dtype=int16), 'sample-000004': array([0, 0, 0, ..., 0, 0, 1], dtype=int16), 'sample-000005': array([0, 0, 0, ..., 0, 0, 0], dtype=int16), 'sample-000006': array([0, 0, 0, ..., 3, 1, 3], dtype=int16), 'sample-000007': array([ 0,  0,  0, ..., -2, -1,  1], dtype=int16), 'sample-000008': array([0, 0, 0, ..., 1, 1, 0], dtype=int16), 'sample-000009': array([0, 0, 0, ..., 0, 0, 0], dtype=int16), 'sample-000010': array([0, 0, 0, ..., 0, 0, 0], dtype=int16), 'sample-000011': array([ 0,  0,  0, ..., -2,  6,  2], dtype=int16), 'sample-000012': array([0, 0, 0, ..., 0, 0, 0], dtype=int16), 'sample-000013': array([-1,  0,  0, ...,  0,  0,  0], dtype=int16), 'sample-000014': array([0, 0, 0, ..., 0, 0, 0], dtype=int16), 'sample-000015': array([0, 0, 0, ..., 0, 0, 0

In [ ]:
path_x='sample.wav'

y=transcription_google(path_x)

path_store="path_to_mutants"

series=all_
explainer = LimeTimeSeriesExplainer(class_names=[0, 1])

#you can change the choice of classifier
exp = explainer.explain_instance(series, classifier_ori_tfidf,key=key, num_samples=100,replacement_method='total_mean')
l=exp.as_list()

new_list=[]
for i in range(len(l)):
  new_list.append(l[i][0])
rank_list_tfidf[key]=new_list
print(rank_list_tfidf.keys())